In [13]:
from PIL import Image
import numpy as np

In [53]:
colors = [
    "000000", "800000", "008000", "808000", "000080", "800080", "008080", "c0c0c0", "808080", "ff0000", "00ff00", "ffff00",
    "0000ff", "ff00ff", "00ffff", "ffffff", "000000", "00005f", "000087", "0000af", "0000d7", "0000ff", "005f00", "005f5f",
    "005f87", "005faf", "005fd7", "005fff", "008700", "00875f", "008787", "0087af", "0087d7", "0087ff", "00af00", "00af5f",
    "00af87", "00afaf", "00afd7", "00afff", "00d700", "00d75f", "00d787", "00d7af", "00d7d7", "00d7ff", "00ff00", "00ff5f",
    "00ff87", "00ffaf", "00ffd7", "00ffff", "5f0000", "5f005f", "5f0087", "5f00af", "5f00d7", "5f00ff", "5f5f00", "5f5f5f",
    "5f5f87", "5f5faf", "5f5fd7", "5f5fff", "5f8700", "5f875f", "5f8787", "5f87af", "5f87d7", "5f87ff", "5faf00", "5faf5f",
    "5faf87", "5fafaf", "5fafd7", "5fafff", "5fd700", "5fd75f", "5fd787", "5fd7af", "5fd7d7", "5fd7ff", "5fff00", "5fff5f",
    "5fff87", "5fffaf", "5fffd7", "5fffff", "870000", "87005f", "870087", "8700af", "8700d7", "8700ff", "875f00", "875f5f",
    "875f87", "875faf", "875fd7", "875fff", "878700", "87875f", "878787", "8787af", "8787d7", "8787ff", "87af00", "87af5f",
    "87af87", "87afaf", "87afd7", "87afff", "87d700", "87d75f", "87d787", "87d7af", "87d7d7", "87d7ff", "87ff00", "87ff5f",
    "87ff87", "87ffaf", "87ffd7", "87ffff", "af0000", "af005f", "af0087", "af00af", "af00d7", "af00ff", "af5f00", "af5f5f",
    "af5f87", "af5faf", "af5fd7", "af5fff", "af8700", "af875f", "af8787", "af87af", "af87d7", "af87ff", "afaf00", "afaf5f",
    "afaf87", "afafaf", "afafd7", "afafff", "afd700", "afd75f", "afd787", "afd7af", "afd7d7", "afd7ff", "afff00", "afff5f",
    "afff87", "afffaf", "afffd7", "afffff", "d70000", "d7005f", "d70087", "d700af", "d700d7", "d700ff", "d75f00", "d75f5f",
    "d75f87", "d75faf", "d75fd7", "d75fff", "d78700", "d7875f", "d78787", "d787af", "d787d7", "d787ff", "d7af00", "d7af5f",
    "d7af87", "d7afaf", "d7afd7", "d7afff", "d7d700", "d7d75f", "d7d787", "d7d7af", "d7d7d7", "d7d7ff", "d7ff00", "d7ff5f",
    "d7ff87", "d7ffaf", "d7ffd7", "d7ffff", "ff0000", "ff005f", "ff0087", "ff00af", "ff00d7", "ff00ff", "ff5f00", "ff5f5f",
    "ff5f87", "ff5faf", "ff5fd7", "ff5fff", "ff8700", "ff875f", "ff8787", "ff87af", "ff87d7", "ff87ff", "ffaf00", "ffaf5f",
    "ffaf87", "ffafaf", "ffafd7", "ffafff", "ffd700", "ffd75f", "ffd787", "ffd7af", "ffd7d7", "ffd7ff", "ffff00", "ffff5f",
    "ffff87", "ffffaf", "ffffd7", "ffffff", "080808", "121212", "1c1c1c", "262626", "303030", "3a3a3a", "444444", "4e4e4e",
    "585858", "626262", "6c6c6c", "767676", "808080", "8a8a8a", "949494", "9e9e9e", "a8a8a8", "b2b2b2", "bcbcbc", "c6c6c6",
    "d0d0d0", "dadada", "e4e4e4", "eeeeee"
]

colors_rgb = [(int(_[:2], 16), int(_[2:4], 16), int(_[4:6], 16))
              for _ in colors]

instruments = ["gold_block", "clay", "packed_ice", "white_wool", "bone_block",
               "iron_block", "soul_sand", "pumpkin", "emerald_block", "hay_block", "glowstone"]


def getCommandArgs(color_idx):
    i = color_idx
    instrument = instruments[i//25]
    note = i % 25
    return "minecraft:"+instrument, "minecraft:note_block[note="+str(note)+"]"


dis = lambda x, y: sum([abs(x[_] - y[_]) for _ in range(3)])


def matchColor(real):
    min_dis = 256 ** 2 * 3
    ret = 0
    for idx, clr in enumerate(colors_rgb):
        now_dis = dis(real, clr)
        if now_dis < min_dis:
            ret = idx
            min_dis = now_dis
    return ret

In [51]:
img = Image.open("64.png")
img = img.convert("RGB")
pic = np.array(img)
H, W = img.height, img.width
target = np.zeros((H, W), dtype=np.int32)
target_img = [[0 for __ in range(W)] for _ in range(H)]

for i in range(H):
    for j in range(W):
        idx = matchColor(pic[i, j, :])
        target[i, j] = idx
        target_img[i][j] = colors_rgb[idx]

In [48]:
target_img = np.array(target_img, dtype=np.uint8)
target_img = Image.fromarray(target_img).convert("RGB")

target_img.save("done_L1.jpg")

In [59]:
with open("cmd.txt", "a+") as f:
    for i in range(H):
        for j in range(W):
            blocks = getCommandArgs(int(target[i, j]))
            f.write(f"/setblock ~{i} ~{0} ~{j} {blocks[0]}\n")
            f.write(f"/setblock ~{i} ~{1} ~{j} {blocks[1]}\n")
